In [1]:
from ideas import import_hook, utils
import token_utils

hi_to_py = {
    "असत": "False",
    "अज्ञात": "None",
    "सत": "True",
    "च": "and",
    "यथा": "as",
    "वा": "assert",
    "सर्ग": "def",
    "यदि": "if",
    "अथवा": "else",    
    "आनय": "import",      
    "ना": "not",
    "या": "or",  
    "सर्गफल": "return",  
    "अपसरण": "break",
    "क्रमशः": "for",
    "में": "in",
    "यदा": "while",
    "क्रमश्रेणि": "range",
    "समीकरण": "lambda",
    # अवधी : "range",
    #"अनुव्रत्ति": "for",
    #"पुनराव्रत्ति": "for",
    # a few builtins useful for beginners    
    #"दर्शय": "print",
    #"श्रेणि": "range",
    #"async": "async",  # do not translate
    #"await": "await",  # as these are not for beginners
    #"is": "is",    
    #"lambda": "lambda",
    #"nonlocal": "nonlocal",
    #"pass": "pass",
    #"raise": "raise",    
    #"from": "from",
    #"global": "global",    
    #"except": "except",
    #"finally": "finally",    
    #"del": "del",
    #"elif": "elif",    
    #"class": "class",
    #"continue": "continue",    
    #"try": "try",
    #"while": "while",
    #"with": "with",
    #"yield": "yield",
    # a few builtins useful for beginners
    #"input": "input",
    #"exit": "exit",  # useful for console
}

hi_to_py_tokens = [tuple(([x.string for x in token_utils.tokenize(token) if x.string != ''], val)) for (token, val) in hi_to_py.items()]
hi_to_py_first = {}

for (i, (hi, py)) in enumerate(hi_to_py_tokens):
    if hi[0] not in hi_to_py_first:
        hi_to_py_first[hi[0]] = []
    hi_to_py_first[hi[0]].append((len(hi), i))
for k, v in hi_to_py_first.items():
    v.sort(reverse=True)

In [2]:
hi_to_py_tokens
hi_to_py_first

{'असत': [(1, 0)],
 'अज': [(5, 1)],
 'सत': [(1, 2)],
 'च': [(1, 3)],
 'यथ': [(2, 4)],
 'व': [(2, 5)],
 'सर': [(3, 12), (3, 6)],
 'यद': [(2, 16), (2, 7)],
 'अथव': [(2, 8)],
 'आनय': [(1, 9)],
 'न': [(2, 10)],
 'य': [(2, 11)],
 'अपसरण': [(1, 13)],
 'क': [(8, 17), (4, 14)],
 'म': [(3, 15)],
 'सम': [(3, 18)]}

In [3]:
hi_to_py_tokens

[(['असत'], 'False'),
 (['अज', '्', 'ञ', 'ा', 'त'], 'None'),
 (['सत'], 'True'),
 (['च'], 'and'),
 (['यथ', 'ा'], 'as'),
 (['व', 'ा'], 'assert'),
 (['सर', '्', 'ग'], 'def'),
 (['यद', 'ि'], 'if'),
 (['अथव', 'ा'], 'else'),
 (['आनय'], 'import'),
 (['न', 'ा'], 'not'),
 (['य', 'ा'], 'or'),
 (['सर', '्', 'गफल'], 'return'),
 (['अपसरण'], 'break'),
 (['क', '्', 'रमश', 'ः'], 'for'),
 (['म', 'े', 'ं'], 'in'),
 (['यद', 'ा'], 'while'),
 (['क', '्', 'रमश', '्', 'र', 'े', 'ण', 'ि'], 'range'),
 (['सम', 'ी', 'करण'], 'lambda')]

In [4]:
def untokenize(tokens): ###
    """
    Converts the output of tokenize.generate_tokens back into a human-readable
    string (that doesn't contain oddly-placed whitespace everywhere).

    **Note:** Unlike tokenize.untokenize(), this function requires the 3rd and
    4th items in each token tuple (though we can use lists *or* tuples).
    """
    out = ""
    last_lineno = -1
    last_col = 0
    for tok in tokens:
        token_string = tok[1]
        start_line, start_col = tok[2]
        end_line, end_col = tok[3]
        # The following two conditionals preserve indentation:
        if start_line > last_lineno:
            last_col = 0
        if start_col > last_col:
            out += (" " * (start_col - last_col))
        out += token_string
        last_col = end_col
        last_lineno = end_line
    return out

import tokenize as py_tokenize

def untokenize(tokens):
    """Return source code based on tokens.
    Adapted from https://github.com/myint/untokenize,
    Copyright (C) 2013-2018 Steven Myint, MIT License (same as this project).
    This is similar to Python's own tokenize.untokenize(), except that it
    preserves spacing between tokens, by using the line
    information recorded by Python's tokenize.generate_tokens.
    As a result, if the original soure code had multiple spaces between
    some tokens or if escaped newlines were used or if tab characters
    were present in the original source, those will also be present
    in the source code produced by untokenize.
    Thus ``source == untokenize(tokenize(source))``.
    Note: if you you modifying tokens from an original source:
    Instead of full token object, ``untokenize`` will accept simple
    strings; however, it will only insert them *as is* without taking them
    into account when it comes with figuring out spacing between tokens.
    """
    words = []
    previous_line = ""
    last_row = 0
    last_column = -1
    last_non_whitespace_token_type = None

    for token in tokens:
        if isinstance(token, str):
            words.append(token)
            continue
        if token.type == py_tokenize.ENCODING:
            continue

        # Preserve escaped newlines.
        if (
            last_non_whitespace_token_type != py_tokenize.COMMENT
            and token.start_row > last_row
        ):
            if previous_line.endswith(("\\\n", "\\\r\n", "\\\r")):
                #pass
                words.append(previous_line[len(previous_line.rstrip(" \t\n\r\\")) :])

        # Preserve spacing.
        if token.start_row > last_row:
            last_column = 0
        if token.start_col > last_column:
            whitelist = set(' \\\n \\\r \\\r\n')
            line_white = ''.join(filter(whitelist.__contains__, token.line[last_column : token.start_col]))
            words.append(line_white)
            #words.append(token.line[last_column : token.start_col])

        words.append(token.string)

        previous_line = token.line
        last_row = token.end_row
        last_column = token.end_col
        if not token.is_space():
            last_non_whitespace_token_type = token.type

    return "".join(words)


In [5]:
def दर्शय(*args):
    res = []
    for arg in args:
        if arg == True:
            res.append("सत")
        elif arg == False:
            res.append("असत")
        elif arg == None:
            res.append("अज्ञात")
        else:
            res.append(arg)
    print(*res)

def विसत्रिताध्यास(fn, li):
    return map(fn, li)
    
def match(stack, tokens):
    #print("matching: ", stack, " *** ", tokens)
    res = False
    for s, t in zip(stack, tokens):
        res = True # if len > 0: res = True
        if s.string != t: return False
    return res
    
def matchIndices(stack, indices):
    for i, index in enumerate(indices):
        if match(stack, hi_to_py_tokens[index[1]][0]):
            return i
    return None

def collapse(curStack, indices, new_tokens):
    matchIndex = matchIndices(curStack, indices)
    #print("eka: ", new_tokens)
    if matchIndex == None:
        new_tokens.extend(curStack)
    else:
        #for x in curStack:
        #    x.string =
        #for i in indices[matchIndex][0]:   
        curStack[0].string = hi_to_py_tokens[indices[matchIndex][1]][1]
        #curStack[0].line = hi_to_py_tokens[indices[matchIndex][1]][0][0]
        curStack[0].end = curStack[-1].end
        new_tokens.append(curStack[0])
        #new_tokens.append(hi_to_py_tokens[indices[matchIndex][1]][1])
        #print("dwa: ", new_tokens)
        #print("putting: ", hi_to_py_tokens[indices[matchIndex][1]][1] , curStack[indices[matchIndex][0]:])
        #print("info: ", indices[matchIndex][0])
        new_tokens.extend(curStack[indices[matchIndex][0]:])
        #print("trini: ", new_tokens)
    
def transform_source१(source, **_kwargs):
    """A simple replacement of 'French Python keyword' by their normal
    English version.
    """
    new_tokens = []
    matches = []
    curMatch = 0
    indices = None
    tokens = token_utils.tokenize(source)
    #for i, x in enumerate(tokens):
    #    print(i, tokens[i:i+1])
    i = 0
    while i < len(tokens):
        pretoken = tokens[i-1] if i > 0 else None
        token = tokens[i]
        if token.string in hi_to_py_first and (type(pretoken) == type(None) or pretoken.end != token.start or (pretoken.type != 1 and pretoken.type != 60)):
            indices = hi_to_py_first[token.string]
            maxLen = indices[0][0]
            curMatch = min(i+maxLen, len(tokens))
            matchIndex = matchIndices(tokens[i:curMatch], indices)
            #print("checking: ", matchIndex, tokens[i:curMatch])
            if matchIndex == None:
                i += 1
            else:
                token.string = hi_to_py_tokens[indices[matchIndex][1]][1]
                token.end = tokens[i+indices[matchIndex][0]].end
                i += indices[matchIndex][0]
                #print("adding transformed token: ", token)
            #matchLen = collapse(tokens[i:curMatch], indices, new_tokens)
            #print("ardhaadhik trini: ", token, new_tokens)
            #val = hi_to_py_tokens[index]
            #token.string = hi_to_py[token.string]
        else:
            i += 1
            #print("chatwari: ", token, new_tokens)
            #print("adding: ", token)
        new_tokens.append(token)
            #print("panch: ", token, new_tokens)
    #new_source = token_utils.untokenize(new_tokens)
    new_source = untokenize(new_tokens)
    #print("final: ", new_source, new_tokens)
    return new_source

def add_hook१(**_kwargs):
    """Creates and adds the import hook in sys.meta_path.
    Uses a custom extension for the exception hook."""
    hook = import_hook.create_hook(
        transform_source=transform_source१,
        hook_name='sanskrit',
        extensions=[".pyhi"],
    )
    return hook

In [6]:
class VistarSyntaxError(Exception):
    """Currently, only raised when a repeat statement has a missing colon."""

    pass

def getTokensList(tokens):
    li = []
    for token in tokens:
        if token.is_space() or token.is_comment():
            continue
        li.append(token)
    return li

def searchString(tokens, s):
    for i, token in enumerate(tokens):
        if token.string == s: return i
    return None

def transform_source२(source, **_kwargs):
    token = "क्रमशः"
    start = [x.string for x in token_utils.tokenize(token) if x.string != '']
    token = "का विस्तार"
    end = [x.string for x in token_utils.tokenize(token) if x.string != '']
    new_tokens = []
    # यदायदा क तदा ख :
    # क का ख में विस्तार :
    # क्रमशः क में ख का विस्तार :
    for tokens in token_utils.get_lines(source):
        # a line of tokens can start with INDENT or DEDENT tokens ...
        tli= getTokensList(tokens)
        #print("orig: ", tokens)
        #ntli = []
        if match(tli, start) and tli[4].end != tli[5].start:
            colonIndex = searchString(tli, ":")
            if colonIndex == None:
                raise VistarSyntaxError(
                    "विस्तार के लिये अंत में : का उपयोग करे|"
                    + f"{tli[0].start_row}\n    {tli[0].line}."
                )
            if match(reversed(tli[:colonIndex]), reversed(end)):
                colonIndex = searchString(tokens, ":")
                vistarIndexRel = searchString(reversed(tokens[:colonIndex]), "क")
                vistarIndex = len(tokens[:colonIndex]) - vistarIndexRel
                #print ("?? this: ", colonIndex, vistarIndexRel, vistarIndex)
                tokens = tokens[:vistarIndex-1] + tokens[colonIndex:]
            #print("tokens: ", tokens)
            #print("joined: ", untokenize(tokens))
            #repeat_index = token_utils.get_first_index(tokens)
            #second_token = tokens[repeat_index + 1]
            #first_token.string = "for %s in %s" % tli[1], tli[3], 
            #last_token.string = "):"
        new_tokens.extend(tokens)
    return untokenize(new_tokens)


def add_hook२(predictable_names=False, **_kwargs):
    """Creates and adds the import hook in sys.meta_path.
    If ``predictable_names`` is set to ``True``, a callback parameter
    passed to the source transformation function will be used to
    create variable loops with predictable names."""
    callback_params = {"predictable_names": predictable_names}
    hook = import_hook.create_hook(
        transform_source=transform_source२,
        callback_params=callback_params,
        hook_name=__name__,
    )
    return hook


if __name__ != '__main__':
    add_hook२()
    add_hook१()

In [7]:
add_hook२()
add_hook१()

<IdeasMetaFinder object for sanskrit>

In [9]:
असत

False

In [10]:
दर्शय(अज्ञात)
अज्ञात

अज्ञात


In [17]:
क्रमशः x में [3, 8, 1]:
    क्रमशः y में [3, 8, 1]:
        pass

In [18]:
क्रमशः y में [1,3] का विस्तार: 
    print(y)

1
3


In [35]:
list(reversed(end))

[':', 'र', 'ा', 'त', '्', 'स', 'ि', 'व', 'ा', 'क']

In [37]:
tli = token_utils.tokenize("क्रमशः क में ख का विस्तार:")
token = "क्रमशः"
start = [x.string for x in token_utils.tokenize(token) if x.string != '']
token = "का विस्तार:"
end = [x.string for x in token_utils.tokenize(token) if x.string != '']
tli= getTokensList(tli)
match(tli, start) and match(reversed(tli), reversed(end))
#list(reversed(tli))

True

In [24]:
tli = token_utils.tokenize("क्रमशः क में ख का विस्तार")
tli
#print(match(tli, "क्रमशः"))# and match(reversed(tli), reversed("काविस्तार:"))

[type=1 (NAME)  string='क'  start=(1, 0)  end=(1, 1)  line='क्रमशः क में ख का विस्तार',
 type=59 (ERRORTOKEN)  string='्'  start=(1, 1)  end=(1, 2)  line='क्रमशः क में ख का विस्तार',
 type=1 (NAME)  string='रमश'  start=(1, 2)  end=(1, 5)  line='क्रमशः क में ख का विस्तार',
 type=59 (ERRORTOKEN)  string='ः'  start=(1, 5)  end=(1, 6)  line='क्रमशः क में ख का विस्तार',
 type=1 (NAME)  string='क'  start=(1, 7)  end=(1, 8)  line='क्रमशः क में ख का विस्तार',
 type=1 (NAME)  string='म'  start=(1, 9)  end=(1, 10)  line='क्रमशः क में ख का विस्तार',
 type=59 (ERRORTOKEN)  string='े'  start=(1, 10)  end=(1, 11)  line='क्रमशः क में ख का विस्तार',
 type=59 (ERRORTOKEN)  string='ं'  start=(1, 11)  end=(1, 12)  line='क्रमशः क में ख का विस्तार',
 type=1 (NAME)  string='ख'  start=(1, 13)  end=(1, 14)  line='क्रमशः क में ख का विस्तार',
 type=1 (NAME)  string='क'  start=(1, 15)  end=(1, 16)  line='क्रमशः क में ख का विस्तार',
 type=59 (ERRORTOKEN)  string='ा'  start=(1, 16)  end=(1, 17)  line='क्रमशः क में

In [8]:
token = "का विस्तार:"
end = [x.string for x in token_utils.tokenize(token) if x.string != '']
end

['क', 'ा', 'व', 'ि', 'स', '्', 'त', 'ा', 'र', ':']

In [17]:
endt = [x for x in token_utils.tokenize(token) if x.string != '']
#getTokensList(endt)
print(match(endt, "का विस्तार:"))
endt

True


[type=1 (NAME)  string='क'  start=(1, 0)  end=(1, 1)  line='का विस्तार:',
 type=59 (ERRORTOKEN)  string='ा'  start=(1, 1)  end=(1, 2)  line='का विस्तार:',
 type=1 (NAME)  string='व'  start=(1, 3)  end=(1, 4)  line='का विस्तार:',
 type=59 (ERRORTOKEN)  string='ि'  start=(1, 4)  end=(1, 5)  line='का विस्तार:',
 type=1 (NAME)  string='स'  start=(1, 5)  end=(1, 6)  line='का विस्तार:',
 type=59 (ERRORTOKEN)  string='्'  start=(1, 6)  end=(1, 7)  line='का विस्तार:',
 type=1 (NAME)  string='त'  start=(1, 7)  end=(1, 8)  line='का विस्तार:',
 type=59 (ERRORTOKEN)  string='ा'  start=(1, 8)  end=(1, 9)  line='का विस्तार:',
 type=1 (NAME)  string='र'  start=(1, 9)  end=(1, 10)  line='का विस्तार:',
 type=54 (OP)  string=':'  start=(1, 10)  end=(1, 11)  line='का विस्तार:']

In [18]:
token_utils.tokenize("क्रमश")

[type=1 (NAME)  string='क'  start=(1, 0)  end=(1, 1)  line='क्रमश',
 type=59 (ERRORTOKEN)  string='्'  start=(1, 1)  end=(1, 2)  line='क्रमश',
 type=1 (NAME)  string='रमश'  start=(1, 2)  end=(1, 5)  line='क्रमश',
 type=4 (NEWLINE)  string=''  start=(1, 5)  end=(1, 6)  line='',
 type=0 (ENDMARKER)  string=''  start=(2, 0)  end=(2, 0)  line='']

In [19]:
token_utils.tokenize("विस्तार")

[type=1 (NAME)  string='व'  start=(1, 0)  end=(1, 1)  line='विस्तार',
 type=59 (ERRORTOKEN)  string='ि'  start=(1, 1)  end=(1, 2)  line='विस्तार',
 type=1 (NAME)  string='स'  start=(1, 2)  end=(1, 3)  line='विस्तार',
 type=59 (ERRORTOKEN)  string='्'  start=(1, 3)  end=(1, 4)  line='विस्तार',
 type=1 (NAME)  string='त'  start=(1, 4)  end=(1, 5)  line='विस्तार',
 type=59 (ERRORTOKEN)  string='ा'  start=(1, 5)  end=(1, 6)  line='विस्तार',
 type=1 (NAME)  string='र'  start=(1, 6)  end=(1, 7)  line='विस्तार',
 type=4 (NEWLINE)  string=''  start=(1, 7)  end=(1, 8)  line='',
 type=0 (ENDMARKER)  string=''  start=(2, 0)  end=(2, 0)  line='']

In [20]:
token_utils.untokenize(token_utils.tokenize("क्रमशः क में ख का विस्तार")[0:1])

'क'

In [29]:
from io import BytesIO
x = BytesIO("अज्ञात".encode('utf-8')).readline
list(py_tokenize.tokenize(x))

[TokenInfo(type=62 (ENCODING), string='utf-8', start=(0, 0), end=(0, 0), line=''),
 TokenInfo(type=1 (NAME), string='अज', start=(1, 0), end=(1, 2), line='अज्ञात'),
 TokenInfo(type=59 (ERRORTOKEN), string='्', start=(1, 2), end=(1, 3), line='अज्ञात'),
 TokenInfo(type=1 (NAME), string='ञ', start=(1, 3), end=(1, 4), line='अज्ञात'),
 TokenInfo(type=59 (ERRORTOKEN), string='ा', start=(1, 4), end=(1, 5), line='अज्ञात'),
 TokenInfo(type=1 (NAME), string='त', start=(1, 5), end=(1, 6), line='अज्ञात'),
 TokenInfo(type=4 (NEWLINE), string='', start=(1, 6), end=(1, 7), line=''),
 TokenInfo(type=0 (ENDMARKER), string='', start=(2, 0), end=(2, 0), line='')]

In [11]:
py_tokenize.tokenize("")

<function tokenize.tokenize(readline)>

In [23]:
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

s = "अज्ञात"
list(tokenize(BytesIO(s.encode()).readline))


[TokenInfo(type=62 (ENCODING), string='utf-8', start=(0, 0), end=(0, 0), line=''),
 TokenInfo(type=1 (NAME), string='अज', start=(1, 0), end=(1, 2), line='अज्ञात'),
 TokenInfo(type=59 (ERRORTOKEN), string='्', start=(1, 2), end=(1, 3), line='अज्ञात'),
 TokenInfo(type=1 (NAME), string='ञ', start=(1, 3), end=(1, 4), line='अज्ञात'),
 TokenInfo(type=59 (ERRORTOKEN), string='ा', start=(1, 4), end=(1, 5), line='अज्ञात'),
 TokenInfo(type=1 (NAME), string='त', start=(1, 5), end=(1, 6), line='अज्ञात'),
 TokenInfo(type=4 (NEWLINE), string='', start=(1, 6), end=(1, 7), line=''),
 TokenInfo(type=0 (ENDMARKER), string='', start=(2, 0), end=(2, 0), line='')]

In [ ]:
अज्ञात

In [ ]:
def

In [ ]:
सर्ग fn():
    return असत

In [ ]:
li = [1, 3, 4]
अमश x in li:
    print(x)

In [ ]:
li = [1, 3, 4]
अक्मश x in li:
    print(x)

In [ ]:
li = [1, 4, 6]
क्रमश x में li का विस्तार:
    print(x)

In [ ]:
li = ['asdf', 34]
fot x it li:
    print(x)

In [ ]:
import token_utils
token_utils.tokenize("for x in li")

In [ ]:
दर्शय('चला', सत)

In [ ]:
x = "dfas"
x.string = "hi"
x

In [ ]:
x = "hi"
y = "hey"
"for %s in %s" % (x, y)

In [ ]:
add_hook()

In [ ]:
token_utils.get